In [1]:
import sys
sys.path.append("./")

import time
from window_adata import *
import os
import sys
import numpy as np
import pandas as pd 
import h5py
import json
import pickle
import anndata
import scanpy as sc
import pytorch_lightning as pl
import matplotlib.pyplot as plt
import glob
import torchvision
import scprep as scp
import anndata as ad
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms

from tqdm import tqdm
from scipy.stats import pearsonr,spearmanr
from scipy import stats
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import TensorBoardLogger
from copy import deepcopy as dcp
from collections import defaultdict as dfd
from sklearn.metrics import adjusted_rand_score as ari_score
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score
from pathlib import Path, PurePath
from typing import Union, Dict, Optional, Tuple, BinaryIO
from matplotlib.image import imread
from scanpy import read_visium, read_10x_mtx
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None
from anndata import read as read_h5ad
from pathlib import Path, PurePath
from typing import Union, Dict, Optional, Tuple, BinaryIO
from matplotlib.image import imread
import pickle

from anndata import (
    AnnData,
    read_csv,
    read_text,
    read_excel,
    read_mtx,
    read_loom,
    read_hdf,)

""" Integrate two visium datasets """
data_dir1 = "./Alex_NatGen_6BreastCancer/"
data_dir2 = "./breast_cancer_10x_visium/"

samps1 = ["1142243F", "CID4290", "CID4465", "CID44971", "CID4535", "1160920F"]
samps2 = ["block1", "block2", "FFPE"]

sampsall = samps1 + samps2
samples1 = {i:data_dir1 + i for i in samps1}
samples2 = {i:data_dir2 + i for i in samps2}

# Marker gene list
gene_list = ["COX6C","TTLL12", "HSP90AB1", "TFF3", "ATP1A1", "B2M", "FASN", "SPARC", "CD74", "CD63", "CD24", "CD81"]

# Highly variable genes
# with open('../1000hvg_common.pkl', 'rb') as f:
#     gene_list = pickle.load(f)
# gene_list = list(gene_list)

# # Load windowed dataset
with open('../10x_visium_dataset_without_window.pickle', 'rb') as f:
    adata_dict0 = pickle.load(f)
for i in samps2:    
    adata_dict0[i].var_names_make_unique()
    
# Define the gridding size
sizes = [4000 for i in range(len(adata_dict0))]

# Split tiles into smaller patches according to gridding size
adata_dict = window_adata(adata_dict0, sizes)


Windowing 1142243F
Num spots:  4784
246
216
222
185
77
247
246
255
247
93
246
247
255
245
94
247
246
255
238
88
130
135
132
140
55
Total:  4787
Windowing CID4290
Num spots:  2714
793
576
1001
344
Total:  2714
Windowing CID4465
Num spots:  1310
345
258
149
558
Total:  1310
Windowing CID44971
Num spots:  1322
491
462
339
30
Total:  1322
Windowing CID4535
Num spots:  1431
564
232
632
3
Total:  1431
Windowing 1160920F
Num spots:  4895
210
251
251
239
83
226
255
232
240
102
231
230
246
247
99
238
246
247
255
93
144
147
164
160
60
Total:  4896
Windowing block1
Num spots:  3798
139
205
219
185
10
169
246
247
255
16
189
230
205
233
0
197
156
241
228
0
72
106
129
124
0
Total:  3801
Windowing block2
Num spots:  3987
224
247
246
229
208
246
247
231
243
207
211
196
221
195
254
205
81
97
108
92
Total:  3988
Windowing FFPE
Num spots:  2518
50
190
188
79
0
169
219
216
189
0
182
215
201
192
0
68
138
159
63
0
0
1
0
0
0
Total:  2519
Windowing 1168993F
Num spots:  4898
244
249
248
244
68
246
247
255
246


In [2]:
# For training
from data_vit import ViT_Anndata

def dataset_wrap(dataloader= True):
    train_sample = list(set(samps1)-set(["1160920F","CID4290"])) # Alex visium samples
    val_sample = ["1160920F","CID4290"] # Alex visium samples
    test_sample = samps2 # 10x visium samples

    tr_name = list(set([i for i in list(adata_dict.keys()) for tr in train_sample if tr in i]))
    val_name = list(set([i for i in list(adata_dict.keys()) for val in val_sample if val in i]))
    te_name = list(set([i for i in list(adata_dict.keys()) for te in test_sample if te in i]))

    trainset = ViT_Anndata(adata_dict = adata_dict, train_set = tr_name, gene_list = gene_list, train=True, flatten=False, ori=True, prune='NA', neighs=4, )
    valset = ViT_Anndata(adata_dict = adata_dict, train_set = val_name, gene_list = gene_list, train=True, flatten=False, ori=True, prune='NA', neighs=4, )
    testset = ViT_Anndata(adata_dict = adata_dict, train_set = te_name, gene_list = gene_list, train=True, flatten=False, ori=True, prune='NA', neighs=4, )

    print("LOADED TRAINSET")
    train_loader = DataLoader(trainset, batch_size=1, num_workers=0, shuffle=True)
    val_loader = DataLoader(valset, batch_size=1, num_workers=0, shuffle=True)
    test_loader = DataLoader(testset, batch_size=1, num_workers=0, shuffle=False)
    
    if dataloader==True:
        return train_loader, val_loader, test_loader
    else:
        return trainset, valset, testset
    
def get_R(data1,data2,dim=1,func=pearsonr):
    adata1=data1.X
    adata2=data2.X
    r1,p1=[],[]
    for g in range(data1.shape[dim]):
        if dim==1:
            r,pv=func(adata1[:,g],adata2[:,g])
        elif dim==0:
            r,pv=func(adata1[g,:],adata2[g,:])
        r1.append(r)
        p1.append(pv)
    r1=np.array(r1)
    p1=np.array(p1)
    return r1,p1



In [3]:
import warnings
warnings.filterwarnings("ignore")
import torch
import random
import numpy as np
import pytorch_lightning as pl
import torch.nn as nn

from gcn import *
from transformer import *
from NB_module import *
from scipy.stats import pearsonr
from torch.utils.data import DataLoader
from copy import deepcopy as dcp
from collections import defaultdict as dfd
from sklearn.metrics import adjusted_rand_score as ari_score
from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score

from torchvision import models, transforms

# set random seed
def setup_seed(seed=12000):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)
    
def get_R(data1,data2,dim=1,func=pearsonr):
    adata1=data1.X
    adata2=data2.X
    r1,p1=[],[]
    for g in range(data1.shape[dim]):
        if dim==1:
            r,pv=func(adata1[:,g],adata2[:,g])
        elif dim==0:
            r,pv=func(adata1[g,:],adata2[g,:])
        r1.append(r)
        p1.append(pv)
    r1=np.array(r1)
    p1=np.array(p1)
    return r1,p1

# HisToGene Training

In [4]:
import os
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from vis_model import HisToGene
from utils import *

In [5]:
# OOD training strategy
tag = '-htg_her2st_785_32_cv'

"""For training only"""
import gc
from data_vit import ViT_Anndata
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import time

start_time = time.time()
"""Training loops"""
seed=12000
epochs=100

"""Load dataset"""
train_loader, val_loader, test_loader = dataset_wrap(dataloader=True)

"""Define model"""
model = HisToGene(n_layers=8, n_genes=len(gene_list), learning_rate=1e-5)
setup_seed(seed)

"""Setup trainer"""
trainer = pl.Trainer(accelerator='auto', max_epochs=epochs, logger=False)
trainer.fit(model, train_loader, val_loader)

"""Save model and clean memory"""
# torch.save(model.state_dict(),f"./model/HisToGene-OOD.ckpt")
trainer.test(model, test_loader)
gc.collect()
del train_loader, model
    
end_time = time.time()
execution_time = end_time - start_time
print("Training time: ", execution_time/3600, " hours")

Eval set:  ['CID4465_0', '1142243F_23', '1142243F_2', 'CID4465_3', '1142243F_20', '1142243F_21', '1142243F_8', '1142243F_10', '1142243F_24', '1142243F_0', '1142243F_1', '1142243F_16', 'CID44971_1', '1142243F_12', '1142243F_18', 'CID4535_0', 'CID4535_2', '1142243F_3', 'CID44971_0', 'CID4465_1', '1142243F_22', '1142243F_15', '1142243F_6', 'CID4465_2', 'CID44971_2', 'CID4535_1', '1142243F_7', '1142243F_17', '1142243F_5', '1142243F_13', '1142243F_14', '1142243F_4', '1142243F_9', '1142243F_11', '1142243F_19', 'CID44971_3']
Loading imgs...
Eval set:  ['CID4290_2', '1160920F_21', 'CID4290_1', '1160920F_8', '1160920F_24', '1160920F_4', '1160920F_19', '1160920F_1', '1160920F_14', '1160920F_17', '1160920F_12', '1160920F_13', '1160920F_7', '1160920F_20', '1160920F_2', 'CID4290_3', '1160920F_5', '1160920F_0', '1160920F_3', '1160920F_11', 'CID4290_0', '1160920F_23', '1160920F_22', '1160920F_16', '1160920F_15', '1160920F_18', '1160920F_6', '1160920F_9', '1160920F_10']
Loading imgs...
Eval set:  ['bl

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type       | Params
-----------------------------------------------
0 | patch_embedding | Linear     | 38.5 M
1 | x_embed         | Embedding  | 131 K 
2 | y_embed         | Embedding  | 131 K 
3 | vit             | ViT        | 67.2 M
4 | gene_head       | Sequential | 14.3 K
-----------------------------------------------
105 M     Trainable params
0         Non-trainable params
105 M     Total params
423.948   Total estimated model params size (MB)
SLURM auto-requeueing enabled. Setting signal handlers.


Epoch 0: 100%|██████████| 36/36 [00:11<00:00,  3.18it/s]                   
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 36/36 [00:02<00:00, 17.64it/s]                
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 36/36 [00:03<00:00, 10.63it/s]                

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
SLURM auto-requeueing enabled. Setting signal handlers.


Testing DataLoader 0: 100%|██████████| 57/57 [00:04<00:00, 12.72it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.8208631277084351     │
└───────────────────────────┴───────────────────────────┘

Training time:  0.029588256345854866  hours


# HisToGene Testing

In [4]:
# For testing

from data_vit import ViT_Anndata

def dataset_wrap(fold = 0, dataloader= True):
    test_sample = sampsall[fold]
    print(f"Test sample: {test_sample}")
    te_name = list(set([i for i in list(adata_dict0.keys()) if test_sample in i]))
    testset = ViT_Anndata(adata_dict = adata_dict0, train_set = te_name, gene_list = gene_list, train=True, flatten=False, ori=True, prune='NA', neighs=4, )
    test_loader = DataLoader(testset, batch_size=1, num_workers=0, shuffle=False)
    if dataloader==True:
        return test_loader
    else:
        return testset

In [5]:
def test(name, model, test_loader):
    model.eval()
    device = 'cpu'
    for patch, center, exp, adj, label, oris, sfs, *_ in test_loader:
        adj=adj.squeeze(0)
        exp=exp.squeeze(0)
        ct = center.squeeze().cpu().numpy()
        gts = exp.squeeze().cpu().numpy()

        """Model inference"""
        gene_exp = model(patch, center).squeeze(0)

        """Put prediction into adata"""
        pred = ad.AnnData(gene_exp.cpu().detach().numpy())
        
        # Marker Genes
        gene_list = ["COX6C","TTLL12", "HSP90AB1", "TFF3", "ATP1A1", "B2M", "FASN", "SPARC", "CD74", "CD63", "CD24", "CD81"]
        
        pred.var_names = gene_list
        pred.obsm['spatial'] = ct
    return pred

In [ ]:
%%time
"""For OOD test only"""
import gc
from vis_model import HisToGene
from data_vit import ViT_Anndata
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import time

start_time = time.time()
"""Training loops"""
for fold in range(9):
    """print sample name"""
    name = sampsall[fold]
    
    """Reproducibility"""
    setup_seed(12000)
    
    """Load dataset"""
    test_loader = dataset_wrap(fold = fold, dataloader= True)
    """Define model"""
    model = HisToGene(n_layers=8, n_genes=len(gene_list), learning_rate=1e-5)
# #     model.load_state_dict(torch.load(f"./model/HisToGene-OOD.ckpt"))

    """Test model performance"""
    pred = test(name, model, test_loader)
    df = pred.var
    df["Gene"] = df.index
    df["Slide"] = name
    df["Method"] = f"HisToGene"
    
    import os
    directory = f"./Results/PCC_table"
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    df.to_csv(f"./Results/PCC_table/{name}_PCC_HisToGene-OOD.csv")
    gc.collect()
    del test_loader, model
end_time = time.time()
execution_time = end_time - start_time
print("Inference time: ", execution_time/3600, " hours")
df


Test sample: 1142243F
Eval set:  ['1142243F']
Loading imgs...
Test sample: CID4290
Eval set:  ['CID4290']
Loading imgs...
Test sample: CID4465
Eval set:  ['CID4465']
Loading imgs...
Test sample: CID44971
Eval set:  ['CID44971']
Loading imgs...
Test sample: CID4535
Eval set:  ['CID4535']
Loading imgs...
Test sample: 1160920F
Eval set:  ['1160920F']
Loading imgs...


In [ ]:
df1 = []
for name in ["1160920F","CID4290"]:
    df = pd.read_csv(f"./Results/PCC_table/{name}_PCC_HisToGene-OOD.csv", index_col=0)
    df1.append(df)
df_train = pd.concat(df1).reset_index()
df_train["Kind"] = "Training dataset (6 samples)"

df1 = []
for name in samps2:
    df = pd.read_csv(f"./Results/PCC_table/{name}_PCC_HisToGene-OOD.csv", index_col=0)
    df1.append(df)
df_test = pd.concat(df1).reset_index()
df_test["Kind"] = "Test dataset (3 samples)"

df_all = pd.concat([df_train, df_test]).reset_index()
df_all.to_csv(f"./Results/PCC_table/HisToGene_visium_MarkerGenes-OOD.csv")
df_all

In [ ]:
fig = plt.figure(figsize=(10,6))
sns.set_style("white")
ax = sns.boxplot(x="Kind", y="Pearson correlation", hue="Method", data=df_all,showfliers=False, orient="v", palette = "Set2")   
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
ax.set(xlabel=None)

plt.xlabel("")
plt.ylabel("Pearson R")
plt.title("Assessment of model generalization capabilities on OOD dataset \n (Predict maker genes only)")
plt.tight_layout()
# plt.savefig("./Figure4.png", dpi=200)
plt.show()
